# Look at data

Navigate to folder with dwi files

In [ ]:
cd /Users/user/Downloads/btc_preop/sub-CON02/ses-preop/dwi

In [ ]:
ls

Convert into format mrtrix3 understands. Combine raw diffusion data with its corresponding b-val and b-vec files

command mrconvert input_file.nii.gz output_file.mif -fslgrad file.bvec file.bval:

In [ ]:
!mrconvert sub-CON02_ses-preop_acq-AP_dwi.nii.gz sub-02_dwi.mif -fslgrad sub-CON02_ses-preop_acq-AP_dwi.bvec sub-CON02_ses-preop_acq-AP_dwi.bval
## ! before command because it's a terminal command on a notebook

Rename files to something more user-friendly:

mv input_file renamed_file

In [ ]:
!mv sub-CON02_ses-preop_acq-AP_dwi.bvec sub-02_AP.bvec
!mv sub-CON02_ses-preop_acq-AP_dwi.bval sub-02_AP.bval
!mv sub-CON02_ses-preop_acq-PA_dwi.bvec sub-02_PA.bvec
!mv sub-CON02_ses-preop_acq-PA_dwi.bval sub-02_PA.bval

Read mif file

In [ ]:
!mrinfo sub-02_dwi.mif

Dimensions: x, y, z, t (4D, last one is time, the number of volumes)
Voxel size: 3D size of each voxel + the repetition time (RT), time it took to acquire each volume 

View bval

Contains a single number per volume and indicates how large the diffusion gradient was applied to data:

In [ ]:
!vi sub-02_AP.bval

View bvec

Contains triplet of number per volume that shows in which directoins the gradients were applied:

In [ ]:
!cat sub-02_AP.bvec

Important that number of bvals and bvecs are the same as number of volumes in dataset. e.g. we can find number of volumes in a dataset 

mrinfo -size file_name.mif | awk '{print $4}'

returns value of the fourth field in the dimension header that corresponds to number of time points aka volumes

In [ ]:
!mrinfo -size sub-02_dwi.mif | awk '{print $4}'

Then compare this with number of bvals and bvecs by using awk to count number of cols in each txt file:

awk '{print NF; exit}' file.bvec
awk '{print NF; exit}' file.bval

They should both return same value we got for n of volumes just above

In [ ]:
!awk '{print NF; exit}' sub-02_AP.bvec
!awk '{print NF; exit}' sub-02_AP.bval

To view files, use mrview command

-Use right-left arrow keys to navigate through volumes and adjust intensity in tools>view options to lower value to see dark volumes

In [ ]:
!mrview sub-02_dwi.mif

# Pre-processing

## Denoise data

dwidenoise input.mif output_name.mif -noise noise.mif (the latter two output noise map)

In [ ]:
!dwidenoise sub-02_dwi.mif sub-02_den.mif -noise noise.mif

Calculate residual to check whether artifacts diproportionately affected anatomy:

mrcalc input.mif output_name_den.mif -subtract residual.mif

In [ ]:
!mrcalc sub-02_dwi.mif sub-02_den.mif -subtract residual.mif

Inspect residual map by opening it. You shouldn't see any anatomical part. If so, that would mean that those part of brain might be corrupted by noise

In [ ]:
!mrview residual.mif

## Distortion correction

diffusion datasets are composed of two different imaging files:
one encoded with primary phase-encoding direction: AP (anterior-posterior)
one encoded with inverse phase-encoding directoin: PA (posterior-anterior), used to unwarp primary images. Create an average that'll cancel out effects of warping

1. Convert PA image into mif format

In [ ]:
!mrconvert sub-CON02_ses-preop_acq-PA_dwi.nii.gz PA.mif

2. Add bvals and bvecs into header
mrconvert command will take PA image consisting of 2 b0 images and use mrmath command to take mean of the two and output it into a .mif file

mrconvert PA_file.mif -fslgrad file_PA.bvec file_PA.bval - | mrmath - mean mean_b0_PA.mif -axis 3

In [ ]:
!mrconvert PA.mif -fslgrad sub-02_PA.bvec sub-02_PA.bval - | mrmath - mean mean_b0_PA.mif -axis 3

3. Extract b-values from AP image and combine the two with mrcat command

 dwiextract file_den.mif - -bzero | mrmath - mean mean_b0_AP.mif -axis 3

 -bzero will extract only those imgs with b0 value. Then take mean of all those and output them into file

In [ ]:
!dwiextract sub-02_den.mif - -bzero | mrmath - mean mean_b0_AP.mif -axis 3

4. Use mrcat to concatenate both mean of b0 imgs of AP and PA imgs

mrcat mean_b0_AP.mif mean_b0_PA.mif -axis 3 b0_pair.mif

In [ ]:
!mrcat mean_b0_AP.mif mean_b0_PA.mif -axis 3 b0_pair.mif

## Unwarp data & remove eddy currents

Now we have everything needed to run the main preprocessing step. To unwarp data and remove eddy currents

dwifslpreproc file_den.mif output_name_den_preproc.mif -nocleanup -pe_dir AP -rpe_pair -se_epi b0_pair.mif -eddy_options " --slm=linear --data_is_shelled"

* -nocleanup will mantain preprocessing folder cause we'll still need some files later
* -pe_dir AP tells main encoding direction
* -rpe_pair -se_epi says that folliwing input file is a pair os spin echo images that were acquired with reverse phase-encoding direction
* -eddy_options fsl command
* --slm=linear useful for data acquired with less than 60 directoins per shell
* --data_is_shelled indicates data acquired with multiple b-vals

In [ ]:
!dwifslpreproc sub-02_den.mif sub-02_den_preproc.mif -nocleanup -pe_dir AP -rpe_pair -se_epi b0_pair.mif -eddy_options " --slm=linear --data_is_shelled"

When it finishes, check out output

mrview preproc_output_den_preproc.mif -overlay.load sub-02_dwi.mif

With this command you open the output and overlay the original data colored in red on top of the preprocessed, to check differences.

In [ ]:
!mrview sub-02_den_preproc.mif -overlay.load sub-02_dwi.mif

## Create a mask

Useful because it restricts analysis only to brain voxels

Remove inhomogeneities in detected the data with dwibiascorrect. This can lead to better mask estimation, however not always the case. That's why you should always check before and after, as with every pre-processing step

dwibiascorrect ants input_preproc.mif output_preproc_unbiased.mif -bias bias.mif

* uses ants option
* see bias fields with bias.mif

In [ ]:
!dwibiascorrect ants sub-02_den_preproc.mif sub-02_den_preproc_unbiased.mif -bias bias.mif

Now we can create mask with dwi2mask command. Which will restrict analysis to voxels located within the brain.

In [ ]:
!dwi2mask sub-02_den_preproc_unbiased.mif mask.mif

To view the mask:

In [ ]:
!mrview mask.mif

# Constrained Spherical Deconvolution

Within each voxel we'll create a basis function from the subject won data by extracting the diffusion signal from representative voxels within the gray and white matter and CSF. We will build a model to estimate what the signal should look like in different orientations and when we apply different b values. If you collect diffusion data with multiple b values, then this approach in MRtrix is called multi-shell multi-tissue (MSMT). There's many, now we will use the dhollander algorithm. 

dwi2response dhollander sub-02_den_preproc_unbiased.mif wm.txt gm.txt csf.txt -voxels voxels.mif

* dhollander: algorithm that deconvolves the fiber orientation distributions (FODs). i.e. it decomposes the diffion signal into a set of smaller individual fiber orientations. The other common algorithm used for this is _tournier_. This is used for single-shell single-tissue (e.g. white matter). Dhollander can be used both for single and multi.
* then we have input
* resulting response functions for each tissue (txt files)
* voxels specifies an output dataset that shows which voxels from the image were used to construct the basis functions for each tissue type

In [ ]:
!dwi2response dhollander sub-02_den_preproc_unbiased.mif wm.txt gm.txt csf.txt -voxels voxels.mif

Now we view the file with mrview command with overlay of the input on top of the deconvoluted output.

In [ ]:
!mrview sub-02_den_preproc_unbiased.mif -overlay.load voxels.mif

Check response function for each tissue type. Output will look like a sphere. This represents what diffusion looks like within that tissue type when a b-value of 0 has been applied, i.e. no diffusion gradient. Pressing left-right arrow keys you can see what response function looks like when different b-values have been applied. Magnitude of sphere becomes smaller when higher b-value are applied. Although higher b-values are more sensitive to changes in diffusion, the overall signal is smaller and more susceptible to noise. 
* In WM sphere tends to flatten like pancake when diffusion gradients are applied, reflecting preferential direction of diffusion along white matter tracts in those voxels. 
* For GM and CSF the basis function remains spherical across all of the b-values. Rate at which sphere gets smaller is different between CSF and GM.

In [ ]:
!shview gm.txt

Now we will use the basis functions to create fiber orientation densities or FODs. These are estimates of the amounts of diffusion in each of the three orthogonal directions and MRtrix can estimate multiple crossing fibers within a single voxel.

dwi2fod msmt_csd sub-02_den_preproc_unbiased.mif -mask mask.mif wm.txt wmfod.mif gm.txt gmfod.mif csf.txt csffod.mif

* dwi2fod command with msmt optiion since it's multi-shell
* input and mask option to restrict analysis to brain voxels only
* tissue type files
* output.mif for each tissue type

In [ ]:
!dwi2fod msmt_csd sub-02_den_preproc_unbiased.mif -mask mask.mif wm.txt wmfod.mif gm.txt gmfod.mif csf.txt csffod.mif

To view these FODs, we will combine them into a single image. Command mrconvert will extract first image from wmfod.mif file, which is the image with b-value of 0. The output of this command is then used as input for mrcat command that will then combine the fod images from all tissue types into a single image: vf.mif

In [ ]:
!mrconvert -coord 3 0 wmfod.mif - | mrcat csffod.mif gmfod.mif - vf.mif

White matter FODs can then be overlaid on this image using mrview with -odf.load_sh command. So we can observe whether the WM FODs do fall within the WM and also if they are allowing the orientation that we would expect

In [ ]:
!mrview vf.mif -odf.load_sh wmfod.mif

## Normalise data for group level analysis

Requires input and output for each tissue type and a mask to restrict to brain voxels

In [ ]:
!mtnormalise wmfod.mif wmfod_norm.mif gmfod.mif gmfod_norm.mif csffod.mif csffod_norm.mif -mask mask.mif

Check output. Make sure color-coding is reasonable and contained in WM.

In [ ]:
!mrview vf.mif -odf.load_sh wmfod_norm.mif

# Create tissue boundary

## Convert t1 to mfi format

In [ ]:
!mrconvert ../anat/sub-CON02_ses-preop_T1w.nii.gz T1.mif

Use command 5ttgen to segment anatomical image into 5 different tissue classes

In [ ]:
!5ttgen fsl T1.mif 5tt_nocoreg.mif

You can then view the output with mrview. Press left and right arriow keys to see the different tissue types like GM, WM and CSF.

In [ ]:
!mrview 5tt_nocoreg.mif

## Co-register anatomical and diffusion weighted images

This ensures that boundaries of the T1 is aligned with dwi

1. Use dwiextract and mrmath to average together the b0 images from the diffusion data. These are the images that look most like the T2-weighted functional scans, since a diffusion gradient wasn't applied during acquisition

* dwiextract takes the pre-processed dwi
* -bzero option extracts b0 images
* \- indicates output should be used as input in the second part of the command
* mrmath then takes these output b0 images and computes the mean along the 3rd axis or the time dimension. I.e., it averages across all volumes in that dataset

In [ ]:
!dwiextract sub-02_den_preproc_unbiased.mif - -bzero | mrmath - mean mean_b0.mif -axis 3

In order to carry out the co-registration between the dwi and t1 images, we need fsl and its flirt command.

First step is to convert both the segmented anatomical image and b0 images we just extracted using mrconvert command

In [ ]:
!mrconvert mean_b0.mif mean_b0.nii.gz
!mrconvert 5tt_nocoreg.mif 5tt_nocoreg.nii.gz

Since flirt works with single 3D image, we will use FSL ROI to extract first volume of segmented dataset which corresponds to GM segmentation

In [ ]:
!fslroi 5tt_nocoreg.nii.gz 5tt_vol0.nii.gz 0 1

We then use flirt command to co-register datasets. 

* This command uses GM segmentation 5tt_vol0.nii.gz as reference image, meaning that it remains stationary. 
* The average b0 images are then moved to find best fit with GM segmentation. 
* Output of this command diff2struct_fsl.mat contains transformation matrix used to overlay the diffusion image on top of GM segmentation

In [ ]:
!flirt -in mean_b0.nii.gz -ref 5tt_vol0.nii.gz -interp nearestneighbour -dof 6 -omat diff2struct_fsl.mat

Now we need to convert transformation matrix into a format readable to mrtrix, using command transformconvert

This command takes the transf matrix we created with fsl and does the co-reg between b0 and 5tt_nocoreg images to generate a transf matrix that canb be read by mrtrix

In [ ]:
!transformconvert diff2struct_fsl.mat mean_b0.nii.gz 5tt_nocoreg.nii.gz flirt_import diff2struct_mrtrix.txt

Since we already have the steps to trasform the diffusion image to anatomical image, we can take inverse of transf matrix to do the opposite: we will co-reg the t1 to diffusion image.

This command takes the 5tt image that hasn't been coreg yet and then applies inverse of the diff2struct transf matrix to create a new file

In [ ]:
!mrtransform 5tt_nocoreg.mif -linear diff2struct_mrtrix.txt -inverse 5tt_coreg.mif

This file can then be loaded into mrview in order to assess quality of co-reg. The overlay color map options specify different color codes for each image that is loaded.

In this case the boundaries before co-reg will be depicted in blue (n. 2), and boundaries after co-reg will be shown in red (n. 1). Make sure to check boundaries in all three views.

You can also use the tool overlay menu to display or hide the different overlays, as well as change opacity of the currently selected figure

Sometimes you can find that the co-reg from flirt does slightly worse or is a little bit off in some places as compared to the initial registration before you applied flirt

In [ ]:
!mrview sub-02_den_preproc_unbiased.mif -overlay.load 5tt_nocoreg.mif -overlay.colourmap 2 -overlay.load 5tt_coreg.mif -overlay.colourmap 1

## Create seed boundary

Last step is to create the seed boundary, separating gray from white matter which we will use to create the seeds for our streamlines. It requires the co-reg dwi and we'll get the output.

In [ ]:
!5tt2gmwmi 5tt_coreg.mif gmwmSeed_coreg.mif

We will check results with mrview to make sure the interface is what we think it should be

In [ ]:
!mrview sub-02_den_preproc_unbiased.mif -overlay.load gmwmSeed_coreg.mif

# Creating the streamlines

Creating streamlines with tkc command

* -act specifies we will use the anatomically segmented image to constrain analysis to WM. This will determine a streamline is valid only if iologically plausible
* -backtrack indicates for the current streamline to go back and run same streamline again if it terminates in a strange place, like CSF
* -maxlengthsets the maximum tract length in voxels that will be permitted
* -cutoff specifies the FOS amplitude for terminating a tract. e.g. a value of .06 would not permit a streamline to go along an FOD that is lower than that number
* -seed_gmwi takes as input the gmwm boundary generated with 5tt
* -nthreads could be used to specify the number of processing cores you wish to use in order to speed up the analysis
* -select indicates how many streamlines to generate
* last two arguments are input and label for output

In [ ]:
!tckgen -act 5tt_coreg.mif -backtrack -seed_gmwmi gmwmSeed_coreg.mif -nthreads 8 -maxlength 250 -cutoff 0.06 -select 10000000 wmfod_norm.mif tracks_10M.tck

To visualize output it is recommended to extract a subset by using tck edit. In this case we will extract 200k of the original 10M.

In [ ]:
!tckedit tracks_10M.tck -number 200k smallerTracks_200k.tck

Output can then be loaded with mrview with -tractography.load, which will automatically overlay the smaller tracks 200k file onto the pre-processed dwi. Check whether the output is legit, namely they should be in WM and color-coded appropriately. e.g. corpus callosum mostly red, corona radiata mostly blue

In [ ]:
!mrview sub-02_den_preproc_unbiased.mif -tractography.load smallerTracks_200k.tck

Although we created an image with reasonable streamlines (aka tractogram), we will still have a problem with some of the wWM tracts being overfitted and others being underfitted. This can be addressed by using the SIFT2 algorithm. Certain tracts can be overprepresented by the amount of streamlines that pass through them, not necessarily because they contain more fibers but because the fibers tend to be all oriented in the same direction.

To counterbalance this overfitting, the command tcksift2 will create a txt file containing weights for each voxel in the brain.

* -act to specificy natomically constraint tractography
* specify n of threads
* and give an input and ouput


In [ ]:
!tcksift2 -act 5tt_coreg.mif -out_mu sift_mu.txt -out_coeffs sift_coeffs.txt -nthreads 8 tracks_10M.tck wmfod_norm.mif sift.txt

# Creating the connectome

A connectome represents the number of streamlines conecting different regions of the brain. First we need to parcelate the brain into nodes.

## Parcellation

You do this with an atlas which assigns each voxel in the brain to a specific node. In this example we use the atlases that come with freesurfer

Before running recon-all, make sure you correctly "source" freesurfer, aka make it available in your shell environment. To be done everytime you want to run freesurfer commands.

In [ ]:
!export FREESURFER_HOME=/usr/local/freesurfer

In [ ]:
!source $FREESURFER_HOME/SetUpFreeSurfer.sh

1. Run the subject's anatomical image through -recon-all

   Make sure you're in the right directory (dwi in this case) and type:

In [ ]:
!SUBJECTS_DIR=`pwd`

This will place the output of recon-all into the dwi directory.

Let's run the recon-all command:

* recon-all is the main command
* -i points to the anatomical image that you will analyze
* input file to be processed
* -s option specifies the subject name, which you can set to whatever you want
* output_name
* -all performs all recon-all steps

In [ ]:
!recon-all -i ../anat/sub-CON02_ses-preop_T1w.nii.gz -s sub-CON02_recon -all

When finished, make sure to use QA procedures to check the output:

https://tinyurl.com/yxsg5p3o

Now we need to convert the labels of the freesurfer parcellation to a format suitable for mrtrix.

the labelconvert command will use the segmentation and parcellation output of fs to create a new parcellated file in.mif format.
* this command uses the aparc+aseg output from fs which contain both the cortical parcellations and subcortical segmentations
* then uses the fs lookup table (LUT.txt file) for the default atlas and convert it into a new table to be read by mrtrix
* final argument is output's label

In [ ]:
!labelconvert sub-CON02_recon/mri/aparc+aseg.mgz $FREESURFER_HOME/FreeSurferColorLUT.txt /usr/local/mrtrix3/share/mrtrix3/labelconvert/fs_default.txt sub-CON02_parcels.mif

## Creating brain connectome

First of all we need to co-register the parcellations as well to the diffusion image:

In [ ]:
!mrtransform sub-CON02_parcels.mif -interp nearest -linear diff2struct_mrtrix.txt -inverse -datatype uint32 sub-CON02_parcels_coreg.mif

The brain connectome would represent the streamlines between each pair of parcellations in the atlas. In this case it'll be 84x84 nodes. 
* -symmetric will make the lower diagonal the same as the upper diagonal
* -scale_invnodevol option will scale connectome by inverse of the size of each node
* -tck_weights_in will wait each connectivity pair by the outfit from sift
* input will be raw 10m tracks we created earlier
* parcels file we just created will be converted into .csv file
* -out_assignment creates a file that can be used for inverse operation of converting a connectome to a track

Command to create the brain connectome with coregistration:

In [ ]:
!tck2connectome -symmetric -zero_diagonal -scale_invnodevol -tck_weights_in sift_1M.txt tracks_10M.tck sub-CON02_parcels_coreg.mif sub-CON02_parcels_coreg.csv -out_assignment assignments_sub-CON02_parcels_coreg.csv

## Viewing connectome

You can view connectome as a matrix in matlab. You need to open matlab and import this file using the importdata command.

You will need to view it as a scaled image so that higher structure connectivity pairs are brighter. 
* To heighten the contrast you can change the scaling of the color map with a vector. In this case 0 is almost bright and 1 is most bright. Keep in mind that matrix is symmetrical.
* Most noticeable feature is a division of the figure in two distinct boxes: representing increased structural connectivity within each hemisphere.
* You will also observe some clusters of brighter voxels, representing higher SC between nearby nodes

Open the lookup table through terminal with vi editor

You will notice that nodes are grouped into distinct areas. The subcortical structures of the LH are nodes 36-42 for example
* Brighter voxels far away from diagnonal can sometimes represent increased SC between homologous regions.
  * if you look up number for row and col on table for voxel k, you'll find it correspond to correlation between node 24 and 73, left and right pre-cuneus

In [ ]:
!vi /usr/local/mrtrix3/share/mrtrix3/labelconvert/fs_default.txt

# Scripting (automating analysis)

You need to do same steps of pre-processing for all of the subjects in your dataset. An alternative to manual could be scripting your analysis to automatize the process.

Create a template that contains the code needed to analyse a single subject and then with a for loop, it will automate it for all subjects.
   * To analyse data with mrtrix we need to run 4 scripts:
       1. Will do all preprocessing
       2. Perform QA checks for major preprocessing outputs
       3. Preprocess SC image using recon-all
       4. Create connectome

## 1. Preprocessing
  
From Andrew's script, first one will do all preprocessing and recquires 7 arguments in precise order:
1. The raw diffusion image;
2. The image acquired with the reverse phase-encoding direction;
3. The bvec file for the data acquired in the AP direction;
4. The bval file for the data acquired in the AP direction;
5. The bvec file for the data acquired in the PA direction;
6. The bval file for the data acquired in the PA direction;
7. The anatomical image


In [ ]:
bash MRtrix_Preproc_AP_Direction.sh [1] [2] [3] [4] [5] [6] [7]

To generalize it to any number of subjects, this code needs to be put into a for loop

In [ ]:
for sub in sub-CON04 sub-CON05; do
    cp *.sh ${sub}/ses-preop/dwi;
    cd ${sub}/ses-preop/dwi;
    bash 01_MRtrix_Preproc_AP_Direction.sh ${sub}_ses-preop_acq-AP_dwi.nii.gz ${sub}_ses-preop_acq-PA_dwi.nii.gz \
    ${sub}_ses-preop_acq-AP_dwi.bvec ${sub}_ses-preop_acq-AP_dwi.bval \
    ${sub}_ses-preop_acq-PA_dwi.bvec ${sub}_ses-preop_acq-PA_dwi.bval \
    ../anat/${sub}_ses-preop_T1w.nii.gz
    cd ../../..;
  done

With this loop we will analyze subject 04 and 05. 

The loop will copy the scripts into each subject's dwi directory, navigate to directory and run code. Only difference is that we replace subject name with variable {sub}, filled by whichever subject name is in the loop.

## 2. QA checks

Same loop to run the QA checks. This allows to check outputs for all the steps

In [ ]:
for sub in sub-CON04 sub-CON05; do        
    cd ${sub}/ses-preop/dwi;     
    bash 02_QC_mrview.sh; 
    cd ../../..;   
  done

## 3. Recon-all

Since tck2connectome requires outcome from recon-all, here's the loop for recon-all.

In [ ]:
 for sub in sub-CON04 sub-CON05; do         
    cd ${sub}/ses-preop/dwi;     
    SUBJECTS_DIR=`pwd`;
    recon-all -i ../anat/${sub}_ses-preop_T1w.nii.gz -s ${sub}_recon -all
    cd ../../..;   
  done

## 4. Tck2connectome

Last loop to run tck2connectome command

In [ ]:
for sub in sub-CON04 sub-CON05; do         
    cd ${sub}/ses-preop/dwi;     
    bash 03_MRtrix_CreateConnectome.sh
    cd ../../..;   
  done

## 5. Visualize

When it's done, open MATLAB to view output

In [ ]:
cd /path/to_folder
connectome = importdata('sub-CON02_parcels.csv');
imagesc(connectome, [0 1])